# Generate Dataset

This notebook geenrates a dataset for a 4 DoF spatial robotic arm described below:
- 4 revolute joints with no limits
- 5 links  each 1 unit long

Our dataset consists of a set of features, $X$, and a set of targets $y$.
- $X$ consists of a 7 values. The first 3 represent a point $(x,y,z)$ and the last 4 represent an orientation in the form of a quaternion $(x,y,z,w)$
- $y$ consists of joint angles, with the amount of features being equal to the amount of joints.

### Adjust Path
In order to use the RobotArm class located in `./utils/robot_arm.py` we need to add this project to our system path.

In [1]:
import sys
import os

# Add the parent directory (project root) relative to the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Current working directory:", os.getcwd())
print("Project root added to sys.path:", project_root)

Current working directory: c:\Users\cjsta\git\inverse_kinematics_prediction\notebooks
Project root added to sys.path: c:\Users\cjsta\git\inverse_kinematics_prediction


### Create a RobotArm object
The RobotArm object defines the structure of a desired arm and can calculate the location of it's end effector.

In [2]:
import numpy as np

from utils.robot_arm import RobotArm

# Create a 4-DOF robot arm with 5 links, each of length 1.0 unit.
num_joints = 4
joint_limits = [(0, 2*np.pi)] * num_joints
link_lengths = [1.0] * (num_joints + 1)

# Instantiate the RobotArm object
robot_arm = RobotArm(num_joints, joint_limits, link_lengths)

# Test forward kinematics
test_angles = [0.5, 0.3, 0.2, 0.1]
pos, quat = robot_arm.forward_kinematics(test_angles)
print("End Effector Position (point x,y,z): ", pos)
print("End Effector Orientation (quaternion x,y,z,w): ", quat)

End Effector Position (point x,y,z):  [ 3.93472065  2.14954769 -1.90423069]
End Effector Orientation (quaternion x,y,z,w):  (np.float64(-0.07311286916773024), np.float64(0.2863331991006688), np.float64(0.2363540298299904), np.float64(0.925637391227236))


### Generate Targets
Our target is the position and orientation of the end effector. We generate a uniformly random set of those. 

In [ ]:
num_samples = 1000  # Parameter to control the number of samples

joint_samples = np.zeros((num_samples, robot_arm.num_joints))

# Generate random angles for each joint within its allowed range.
for i, (low, high) in enumerate(robot_arm.joint_limits):
    joint_samples[:, i] = np.random.uniform(low, high, size=num_samples)

print("Generated random joint samples shape:", joint_samples.shape)

Generated random joint samples shape: (1000, 4)


array([[0.21141165, 0.06279582, 1.29609117, 6.27870023],
       [0.07860028, 4.95024723, 4.73714942, 0.62200772],
       [5.40065228, 0.72907796, 2.76707244, 5.86556121],
       ...,
       [1.62143803, 5.38942214, 1.89914489, 0.88074172],
       [3.47995185, 2.36739053, 5.07178472, 4.12369753],
       [0.81644348, 2.30549179, 4.2330438 , 3.46932546]], shape=(1000, 4))

### Generate Features
Our features are the position and orientation of the end effector.



## CONSIDER MAKING A SPATIAL AND PLANAR RobotArm CLASS